# Imports

In [ ]:
import pandas as pd
import numpy as np
import os
import gc
import sys 

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from utils import clean_database, perform_voting_feature_selection, perform_transformation_feature_selection, perform_voting_feature_transformation
pd.set_option('display.max_columns', None)

# Choose the model RF/MLP, you should also change the eval_model in utils.py to the same model
model_name = 'RF'
preprocessing_dir = os.path.join('figures','preprocessing', model_name)
db_path = os.path.join('..', 'data', 'datasets', '5G NIDD', 'Combined.csv')

# Data cleaning and splitting

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test, output_encoder = clean_database(db_path, preprocessing_dir, do_scale=True, fix_skewness=False)
gc.collect()


In [ ]:
# Print the number of remaining features
print(f"Remaining features count: {X_train.shape[1]}")

# Print the list of remaining feature names
print("Remaining features list:")
print(X_train.columns.tolist())

# Feature selection methods loop

In [ ]:

n_features_list = [35, 30, 25, 20, 15, 10, 5, 1]
# Helper to save DataFrame or Numpy array to CSV
def save_to_csv(data, full_path):
    if isinstance(data, pd.DataFrame):
        data.to_csv(full_path, index=False)
    else:
        # Convert numpy array to DataFrame
        pd.DataFrame(data).to_csv(full_path, index=False)
        
# 1. Loop through each configuration
for n in n_features_list:
    print(f"\n{'='*80}")
    print(f"STARTING PIPELINE: {n} FEATURES")
    print(f"{'='*80}")
    
    # --- A. Setup Version and Directory ---
    version = f"{n}F"
    save_dir = os.path.join('..', 'data frames', 'main', model_name, f'{n} features')
    
    # Create the directory if it doesn't exist
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"[Setup] Created directory: {save_dir}")
    else:
        print(f"[Setup] Directory exists: {save_dir}")

    # --- B. Perform Feature Selection ---
    # We pass the ORIGINAL full datasets (X_train, etc.)
    print(f"[Step 1] Running Voting Feature Selection...")
    selected_features, comparison_results = perform_voting_feature_selection(
        X_train, y_train, X_val, y_val, 
        n_features=n, 
        sample_size=None, 
        file_path=preprocessing_dir, # Keeps selection plots in the main preprocessing folder
        version=version
    )
    
    # Save the list of selected features to a text file
    with open(os.path.join(save_dir, f'selected_features_{version}.txt'), 'w') as f:
        f.write('\n'.join(selected_features))

    # --- C. Subset the Data ---
    # CRITICAL: We create COPIES (sub) instead of overwriting X_train.
    # This preserves the original X_train for the next iteration of the loop.
    print(f"[Step 2] Dropping non-selected features...")
    X_train_sub = X_train[selected_features].copy()
    X_val_sub = X_val[selected_features].copy()
    X_test_sub = X_test[selected_features].copy()

    # --- D. Save the Dataframes (Skipping Preprocessing/Sampling) ---
    print(f"[Step 3] Saving dataframes to {save_dir} (Skipping Sampling)...")
    
    save_to_csv(X_train_sub, os.path.join(save_dir, f'X_train_{version}.csv'))
    save_to_csv(X_val_sub, os.path.join(save_dir, f'X_val_{version}.csv'))
    save_to_csv(X_test_sub, os.path.join(save_dir, f'X_test_{version}.csv'))
    save_to_csv(y_train, os.path.join(save_dir, f'y_train_{version}.csv'))
    save_to_csv(y_val, os.path.join(save_dir, f'y_val_{version}.csv'))
    save_to_csv(y_test, os.path.join(save_dir, f'y_test_{version}.csv'))

    # --- F. Save Label Classes ---
    # Saving the classes is crucial for decoding predictions later
    np.save(os.path.join(save_dir, f'label_classes_{version}.npy'), output_encoder.classes_)
    
    print(f"COMPLETED: {n} features pipeline finished.\n")


# Transformation Methods

In [ ]:
# --- Transformation Methods ---
# We use the voting function to compare PCA/LDA with different components
# and automatically select the best one based on validation accuracy.

print(f"\n{'='*80}")
print(f"STARTING FEATURE TRANSFORMATION SELECTION")
print(f"{'='*80}")

# 1. Run Voting for Transformation Methods
# This will compare PCA and LDA with 0.99, 0.95, and 0.90 variance/components
best_transformation, transformation_results = perform_voting_feature_transformation(
    X_train, y_train, X_val, y_val,
    n_components_options=[0.99, 0.95, 0.90],
    sample_size=None,
    file_path=preprocessing_dir,
    version="transformation_comparison"
)

if best_transformation:
    best_method = best_transformation['method']
    best_n_comp = best_transformation['n_components']
    
    print(f"\n[Result] Best Transformation: {best_method.upper()} with n_components={best_n_comp}")
    
    # 2. Apply the Best Transformation to all datasets
    # We need to re-run the transformation with the best parameters to get the final datasets
    # Note: We pass X_test to get it transformed as well
    X_train_trans, X_val_trans, X_test_trans, exec_time = perform_transformation_feature_selection(
        X_train, y_train, X_train, X_val, X_test, # Fit on X_train
        n_components=best_n_comp, 
        method=best_method
    )
    
    # 3. Setup Directory for the best transformation
    version = f"{best_method.upper()}_{best_n_comp}"
    save_dir = os.path.join('..', 'data frames', 'main', model_name, version)
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"[Setup] Created directory: {save_dir}")
        
    # 4. Save Transformed Dataframes
    print(f"[Step 3] Saving best transformed dataframes to {save_dir}...")
    save_to_csv(X_train_trans, os.path.join(save_dir, f'X_train_{version}.csv'))
    save_to_csv(X_val_trans, os.path.join(save_dir, f'X_val_{version}.csv'))
    save_to_csv(X_test_trans, os.path.join(save_dir, f'X_test_{version}.csv'))
    
    # Save targets (unchanged)
    save_to_csv(y_train, os.path.join(save_dir, f'y_train_{version}.csv'))
    save_to_csv(y_val, os.path.join(save_dir, f'y_val_{version}.csv'))
    save_to_csv(y_test, os.path.join(save_dir, f'y_test_{version}.csv'))
    
    # Save Label Classes
    np.save(os.path.join(save_dir, f'label_classes_{version}.npy'), output_encoder.classes_)
    
    print(f"COMPLETED: Best transformation pipeline finished.\n")
else:
    print("No suitable transformation method found.")
